In [6]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
%matplotlib inline

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [8]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
shape = (64, 64)
channel = 3
data = generate_data_set_for_training(shape, channel)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [10]:
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Try scratched Vgg

In [15]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [12]:
SHAPE = (64, 64)
CHANNEL = 3
    
data = generate_data_set_for_training(shape=SHAPE, channel=CHANNEL)
# Extract Train, val from data
X_train = data['X_train']
X_val = data['X_val']
# Format as the input of VGG
y_train = list(data['y_train'].T)
y_val = list(data['y_val'].T)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [16]:
_,row, col,channel = X_train.shape
digLen = 5 # including category 0
numDigits = 11
epochs = 50
batch_size = 64

vgg16Model = VGG16(include_top = False,
                   weights = None)
vgg16Model.summary()
ptInput = keras.Input(shape = (row,col,channel), name  = 'vgg16Scratch')
vgg16 = vgg16Model(ptInput)

# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(input)
# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)

vgg16 = Flatten()(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
# vgg16 = Dense(1000, activation='relu')(vgg16)
vgg16 = Dropout(0.5)(vgg16)

numd_SM = Dense(digLen,    activation='softmax',name = 'num')(vgg16)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(vgg16)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(vgg16)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(vgg16)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(vgg16)
numB_SM = Dense(2,         activation='softmax',name = 'nC')(vgg16)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM, numB_SM]

vgg16 = keras.Model(inputs = ptInput, outputs = out)

callback = []
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/vgg16.classifier.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 3,
                                              cooldown= 0,
                                              min_lr = 0.000001)
# tb = keras.callbacks.TensorBoard(log_dir='logs', write_graph=True, write_images=True)
es = keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                   min_delta=0.00000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)
vgg16.summary()

vgg16.compile(loss = 'sparse_categorical_crossentropy',
              optimizer= optim,
              metrics=  ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [18]:
vgg16History = vgg16.fit(x = X_train,
                         y = y_train,
                         batch_size = batch_size,
                         epochs=epochs,
                         verbose=1,
                         shuffle = True,
                         validation_data = (X_val, y_val),
                         callbacks = callback)

print(vgg16History.history.keys())
modName = 'vgg16_Scratch'
print(vgg16History.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50


FailedPreconditionError: Attempting to use uninitialized value training/Adam/Variable_33
	 [[{{node training/Adam/Variable_33/read}} = Identity[T=DT_FLOAT, _class=["loc:@training/Adam/Assign_133"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/Variable_33)]]
	 [[{{node loss/dig3_loss/Mean_2/_443}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3315_loss/dig3_loss/Mean_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance

evaluate_model_performance(fitted_model=vgg16History, 
                           model_name = modName,
                           data=data,
                           model=vgg16
                          )